In [1]:
import os
os.chdir(os.path.dirname(os.getcwd()))

## init combined robot config

In [2]:
from pkg.controller.combined_robot import *
from pkg.project_config import *

crob = CombinedRobot(robots_on_scene=[
    RobotConfig(0, RobotType.indy7, None,
                INDY_IP),
    RobotConfig(1, RobotType.panda, None,
                "{}/{}".format(PANDA_REPEATER_IP, PANDA_ROBOT_IP))]
              , connection_list=[False, False])

connection_list
[False, False]


## init stereo aruco detector

In [3]:
from pkg.marker_config import *
from pkg.detector.aruco.stereo import ArucoStereo
from pkg.detector.camera.realsense import RealSense
from pkg.detector.camera.kinect import Kinect
aruco_map = get_aruco_map()
stereo = ArucoStereo(aruco_map, [Kinect(), RealSense()])
stereo.initialize()
stereo.calibrate()

Device configuration: 
	color_format: 3 
	(0:JPG, 1:NV12, 2:YUY2, 3:BGRA32)

	color_resolution: 5 
	(0:OFF, 1:720p, 2:1080p, 3:1440p, 4:1536p, 5:2160p, 6:3072p)

	depth_mode: 3 
	(0:OFF, 1:NFOV_2X2BINNED, 2:NFOV_UNBINNED,3:WFOV_2X2BINNED, 4:WFOV_UNBINNED, 5:Passive IR)

	camera_fps: 2 
	(0:5 FPS, 1:15 FPS, 2:30 FPS)

	synchronized_images_only: False 
	(True of False). Drop images if the color and depth are not synchronized

	depth_delay_off_color_usec: 0 ms. 
	Delay between the color image and the depth image

	wired_sync_mode: 0
	(0:Standalone mode, 1:Master mode, 2:Subordinate mode)

	subordinate_delay_off_master_usec: 0 ms.
	The external synchronization timing.

	disable_streaming_indicator: False 
	(True or False). Streaming indicator automatically turns on when the color or depth camera's are in use.


Start streaming


[(array([[1.82983423e+03, 0.00000000e+00, 1.91572046e+03],
         [0.00000000e+00, 1.82983423e+03, 1.09876086e+03],
         [0.00000000e+00, 0.00000000e+00, 1.00000000e+00]]),
  array([ 7.09966481e-01, -2.73409390e+00,  1.45804870e-03, -3.24774766e-04,
          1.44911301e+00,  5.84310412e-01, -2.56374550e+00,  1.38472950e+00])),
 (array([[1.39560388e+03, 0.00000000e+00, 9.62751587e+02],
         [0.00000000e+00, 1.39531934e+03, 5.47687012e+02],
         [0.00000000e+00, 0.00000000e+00, 1.00000000e+00]]),
  array([0., 0., 0., 0., 0.])),
 array([[ 0.83051735, -0.03029965, -0.556168  ,  0.64800465],
        [ 0.17347726,  0.96292555,  0.20659147, -0.14270642],
        [ 0.5292887 , -0.26806033,  0.80498266,  0.07150432],
        [ 0.        ,  0.        ,  0.        ,  1.        ]],
       dtype=float32)]

## init scene builder

In [4]:
from pkg.geometry.builder.scene_builder import SceneBuilder
s_builder = SceneBuilder(stereo)
s_builder.reset_reference_coord(ref_name="floor")

## get ghnd with detected robot config

In [5]:
xyz_rpy_robots = s_builder.detect_items(level_mask=[DetectionLevel.ROBOT])
crob.update_robot_pos_dict(xyz_rpy_robots=xyz_rpy_robots)
gscene = s_builder.create_gscene(crob)

Unknown tag "hardwareInterface" in /robot[@name='custom_robots']/transmission[@name='indy0_tran0']/actuator[@name='indy0_motor0']
Unknown tag "hardwareInterface" in /robot[@name='custom_robots']/transmission[@name='indy0_tran1']/actuator[@name='indy0_motor1']
Unknown tag "hardwareInterface" in /robot[@name='custom_robots']/transmission[@name='indy0_tran2']/actuator[@name='indy0_motor2']
Unknown tag "hardwareInterface" in /robot[@name='custom_robots']/transmission[@name='indy0_tran3']/actuator[@name='indy0_motor3']
Unknown tag "hardwareInterface" in /robot[@name='custom_robots']/transmission[@name='indy0_tran4']/actuator[@name='indy0_motor4']
Unknown tag "hardwareInterface" in /robot[@name='custom_robots']/transmission[@name='indy0_tran5']/actuator[@name='indy0_motor5']


Please create a subscriber to the marker
publication OK
published: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


## add environment

In [6]:
gtem_dict = s_builder.detect_and_register(level_mask=[DetectionLevel.ENVIRONMENT])

Please create a subscriber to the marker


In [7]:
# add cam poles
ptems = s_builder.add_poles({"cam0": s_builder.ref_coord_inv[:3,3], "cam1":np.matmul(s_builder.ref_coord_inv, stereo.T_c12)[:3,3]})

In [8]:
gtems = s_builder.add_robot_geometries(color=(0,1,0,0.5), display=True, collision=True, exclude_link=["panda1_link7"])

## add movable

In [9]:
gtem_dict = s_builder.detect_and_register(level_mask=[DetectionLevel.MOVABLE])

## init planning scene

In [10]:
from pkg.planning.scene import PlanningScene
pscene = PlanningScene(gscene, combined_robot=crob)

## Register binders

In [11]:
from pkg.planning.constraint.constraint_action import Gripper2Tool, PlacePlane

In [12]:
gscene.create_safe(gtype=GEOTYPE.SPHERE, name="grip0", link_name="indy0_tcp", 
                 dims=(0.01,)*3, center=(0,0,0.14), rpy=(-np.pi/2,0,0), color=(1,0,0,1), display=True, collision=False, fixed=True)
gscene.create_safe(gtype=GEOTYPE.SPHERE, name="grip1", link_name="panda1_hand", 
                 dims=(0.01,)*3, center=(0,0,0.112), rpy=(-np.pi/2,0,0), color=(1,0,0,1), display=True, collision=False, fixed=True)

In [13]:
pscene.create_binder(bname="grip0", gname="grip0", _type=Gripper2Tool, point=(0,0,0), rpy=(0,0,0))
pscene.create_binder(bname="grip1", gname="grip1", _type=Gripper2Tool, point=(0,0,0), rpy=(0,0,0))
pscene.create_binder(bname="floor", gname="floor", _type=PlacePlane)
pscene.create_binder(bname="goal_bd", gname="goal", _type=PlacePlane, point=(0,0,0.02), rpy=(0,0,0))

## add objects

In [14]:
from pkg.planning.constraint.constraint_object import BoxAction

In [15]:
box1 = pscene.create_object(oname="box1", gname="box1", _type=BoxAction, hexahedral=True)
box2= pscene.create_object(oname="box2", gname="box2", _type=BoxAction, hexahedral=True)

## Register object binders

In [16]:
box1.register_binders(pscene, PlacePlane)
box2.register_binders(pscene, PlacePlane)

## Motion Planning

### moveit

In [17]:
from pkg.planning.motion.moveit.moveit_planner import MoveitPlanner
from pkg.planning.scene import State
mplan = MoveitPlanner(pscene, enable_dual=True)
mplan.update_gscene()

In [18]:
from pkg.planning.constraint.constraint_common \
            import sample_redundancy, combine_redundancy

obj, handle, binder = "box2", "top_g", "grip1"
from_state = pscene.get_state(crob.home_pose)
to_state = pscene.make_goal_state(from_state, obj=obj, handle=handle, binder=binder)

In [19]:
for _ in range(10):
    redundancy = sample_redundancy(
        combine_redundancy(
            pscene.object_dict[obj].action_points_dict[handle], 
            pscene.binder_dict[binder]))
    traj, lastQ, error, success, binding_list = mplan.plan_transition(from_state, to_state, 
                                                        redundancy_dict={obj: redundancy})
    print(success)
    if success:
        print(lastQ)
        break

False
False
True
[ 0.          0.         -1.57079633  0.         -1.57079633  0.
  1.82025827 -1.12797858 -2.69510271 -1.5150672  -2.70393131  2.185727
  0.181556  ]


In [20]:
pscene.gscene.show_motion(traj, period=0.05)
for bd in binding_list:
    pscene.rebind(bd, list2dict(lastQ, crob.joint_names))
node, obj_pos_dict = pscene.get_object_state()
end_state = State(node, obj_pos_dict, lastQ, pscene)

In [24]:
obj, handle, binder = "box2", "front_g", "grip0"
from_state2 = end_state.copy(pscene)
to_state2 = pscene.make_goal_state(from_state2, obj=obj, handle=handle, binder=binder)

In [25]:
for _ in range(30):
    redundancy = sample_redundancy(
        combine_redundancy(
            pscene.object_dict[obj].action_points_dict[handle], 
            pscene.binder_dict[binder]))
    traj, lastQ, error, success, binding_list = mplan.plan_transition(from_state2, to_state2, 
                                                        redundancy_dict={obj: redundancy}, timeout=1)
    print(success)
    if success:
        break

indy0_panda1
[-0.          1.57079633 -0.          1.57079633 -0.         -0.
  1.82025827 -1.12797858 -2.69510271 -1.5150672  -2.70393131  2.185727
  0.181556  ]
True


In [26]:
pscene.gscene.show_motion(traj, period=0.05)

## Disconnect stereo

In [20]:
stereo.disconnect()